<img src="https://raw.githubusercontent.com/Mauritas99/Proyect_images/refs/heads/main/Recommendation_model.jpg">

><h2>1. Importar librerias</h2>

In [ ]:
pip install lightfm

In [ ]:
pip install category_encoders

In [4]:
import pandas as pd
from lightfm import LightFM
from category_encoders import TargetEncoder
from scipy.sparse import coo_matrix,csr_matrix
from sklearn.preprocessing import OneHotEncoder
from lightfm.evaluation import precision_at_k

In [5]:
df_wines = pd.read_csv("../1.1_Data_limpia/df_wines_limpio_2.csv")

<h5><li style="font-size:1.3rem; top:2px;"><em>Sistema de Recomendación de Vinos</em></li></h5>
    <ul>
        <li style="font-size:1.2rem; margin-top:10px;">[x] Basado en contenido: Recomendaciones en función de la variedad, país, etc.
        </li>
        <li style="font-size:1.2rem; margin-top:10px;">[x] Colaborativo: Recomendaciones basadas en la similitud entre puntuaciones de usuarios.
        </li>
    </ul>
    <br>

><h2>2. Preparar dataset</h2>

In [6]:
df_wines_recom = df_wines.copy()

In [7]:
# Ordenamos columnas
columnas_ordenadas = ['country','province','region_1', 'region_2','title', 'description', 'designation', 'variety', 'winery','points', 'price','price_categ','taster_name', 'taster_twitter_handle',]
df_wines_recom = df_wines_recom[columnas_ordenadas]

In [8]:
for column in df_wines_recom.columns:
    count = df_wines_recom[column].value_counts()
    print(f"frecuencia mas alta para {column}: {count.idxmax()}") # Retiramos aquellas filas con cantidad alta de datos desconocidos.


frecuencia mas alta para country: US
frecuencia mas alta para province: California
frecuencia mas alta para region_1: Sin dato
frecuencia mas alta para region_2: No posee sub-región
frecuencia mas alta para title: Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County)
frecuencia mas alta para description: This opens with subtle aromas of wild flower and a whiff of exotic fruit. The bright fruity palate offers white peach, apple and a hint of honeydew melon. It's straightforward and refreshing.
frecuencia mas alta para designation: Desconocida
frecuencia mas alta para variety: Pinot Noir
frecuencia mas alta para winery: Wines & Winemakers
frecuencia mas alta para points: 88
frecuencia mas alta para price: 35
frecuencia mas alta para price_categ: Premium
frecuencia mas alta para taster_name: Desconocido
frecuencia mas alta para taster_twitter_handle: Desconocido


In [9]:
umbral = 2 # Cantidad de nulos maximos por fila

df_recom_replaced = df_wines_recom.replace(["No posee sub-región","Sin dato","Desconocido","Desconocida","Procedencia desconocida"],pd.NA)
df_recom_filtrado = df_recom_replaced[df_recom_replaced.isna().sum(axis=1) <= umbral]
df_recom_filtrado = df_recom_filtrado.fillna("Sin dato")

df_wines_recom = df_recom_filtrado # Se le retiro un 20k de filas al dataset.

<h4><em>2.1 Generar dataset vinos y usuarios.</em></h4>

In [11]:
df_wines_caract = df_wines_recom.drop(columns=["description","region_2","taster_twitter_handle"]) # Dataframe caracteristicas de vinos.
df_wines_caract.reset_index(inplace=True)

In [12]:
wine_ids = {title: i for i, title in enumerate(df_wines_recom['title'].unique())} # Asignar un ID a cada vino.
taster_ids = {name: i for i, name in enumerate(df_wines_recom['taster_name'].unique())} # Asignar ID a cada sommelier.

In [13]:
# Agregamos los id al dataset.
df_wines_caract['wine_id'] = df_wines_caract['title'].map(wine_ids)
df_wines_caract['taster_id'] = df_wines_caract['taster_name'].map(taster_ids)

In [14]:
# Generamos el dataset de sommeliers.
df_tasters = pd.DataFrame({
    'taster_id': df_wines_caract['taster_name'].map(taster_ids),
    'wine_id': df_wines_caract['title'].map(wine_ids),
    'points': df_wines_caract['points']
})

In [15]:
# Columnas a codificar

col_cat_target = ["country", "province", "region_1", "variety", "winery", "taster_name","designation"]
col_num_target = ["points"]

encoder = TargetEncoder(cols=col_cat_target)


encoded_data = encoder.fit_transform(df_wines_caract[col_cat_target], df_wines_caract[col_num_target]) # Debido al tamaño que generaria OHE, se reemplaza por valor numerico de la categoria teniendo en cuenta "points"
df_wines_caract[col_cat_target] = encoded_data[col_cat_target]

category_mapping = {}
for col in col_cat_target: # Genera un diccionario para encontrar el valor str asociado al valor numerico de targetEncoder.
    unique_categories = df_wines_recom[col].unique()

    temp_df = pd.DataFrame({c: ['placeholder'] * len(unique_categories) for c in col_cat_target})

    temp_df[col] = unique_categories
    encoded_values = encoder.transform(temp_df)[col].tolist()
    category_mapping[col] = dict(zip(unique_categories, encoded_values))

In [16]:
encoded_country = category_mapping["taster_name"]["Kerin O’Keefe"] # Mediante este mapa se obtiene el valor numerico al que corresponde una categoria de una variable codeada.
encoded_country

88.89763698276752

<ul>
  <li><em>DataFrame de caracteristica de vinos: <b>df_wines_caract</b></li>
  <li>Dataframe interacciones de sommeliers: <b>df_tasters</b></em></li>
</ul>

<h4><em>2.2 Crear matriz de interacciones</em></h4>

In [17]:
interactions = coo_matrix((df_tasters['points'],
                            (df_tasters['taster_id'], df_tasters['wine_id'])))
interactions_matrix = csr_matrix(interactions) # Matriz de interacciones.

In [18]:
interactions_matrix.shape

(20, 98454)

<h4><em>2.3 Crear matriz de características</em></h4>

In [19]:
df_wines_caract_matrix = df_wines_caract.drop(columns=["points","taster_id","price","title"]) # Retiramos caracteristicas que no aportan a la matriz.

In [20]:
# Codificar la variable price_categ con OHE
ohe = OneHotEncoder()
price_categ_ohe = ohe.fit_transform(df_wines_caract_matrix[["price_categ"]])
price_categ_ohe_df = pd.DataFrame(price_categ_ohe.toarray(), columns=ohe.get_feature_names_out(["price_categ"]))

df_wines_caract_matrix = pd.concat([df_wines_caract_matrix, price_categ_ohe_df], axis=1)
df_wines_caract_matrix = df_wines_caract_matrix.drop(columns=["price_categ"])


In [21]:
item_features = df_wines_caract_matrix
features_matrix = csr_matrix(item_features) # Matriz de caracteristicas.


<h4><em>2.4 Diccionario</em></h4>

In [22]:
wine_id_to_title = dict(zip(df_wines_caract['wine_id'], df_wines_caract['title'])) # Obtener nombre del vino segun su Id.

><h2>3. Modelo LightFM</h2>

In [ ]:
model_lightfm = LightFM(loss='warp', no_components=33, learning_rate=0.05, random_state=42)

model_lightfm.fit(interactions=interactions_matrix, epochs=10, num_threads=2) # Entrenar modelo.

<h4><em>3.1 Evaluar modelo.</em></h4>

In [ ]:
k = 10
print('Precision del entrenamiento para valor de k={}:\t{:.4f}'.format(k, precision_at_k(model_lightfm, interactions_matrix, k=k).mean()))

Precision del entrenamiento para valor de k=10:	0.8700


><h2>4. Funciones de recomendacion.</h2>

In [ ]:
def recomendar_por_pais(pais:str,model:LightFM=model_lightfm,features_matrix:csr_matrix=features_matrix,wine_id_to_title:dict=wine_id_to_title,category_mapping:dict=category_mapping,n=10) -> list:
    """
    Recomienda vinos por país de procedencia.
    """

    # 1. Obtenemos el ID del país usando el diccionario category_mapping
    pais_id = category_mapping["country"][pais]

    # 2. Buscamos los vinos con ese país de procedencia en features_matrix
    country_index = item_features.columns.get_loc("country")  # Obtener índice de la columna "country"
    wine_ids_pais = features_matrix[:, country_index].toarray().flatten() == pais_id  # Buscar ID del país

    # 3. Generamos las predicciones para todos los vinos con ese país de procedencia
    scores = model.predict(user_ids=0, item_ids=np.arange(features_matrix.shape[0]), item_features=features_matrix)

    # 4. Filtramos por los vinos que coincidan con el ID del país
    scores_pais = scores[wine_ids_pais]

    # 5. Ordenamos las predicciones y seleccionamos los n mejores
    top_items_pais = np.argsort(-scores_pais)[:n]

    # 6. Obtenemos los nombres de los vinos con su ID
    wine_ids_for_country = item_features.index[wine_ids_pais] # obtener los wine_ids que corresponden al país
    recommendations = [wine_id_to_title[item_features.loc[wine_ids_for_country[item], 'wine_id']] for item in top_items_pais]

    return recommendations

In [ ]:
recomendar_por_pais(pais="Argentina")

['Finca Las Moras 2010 Pedernal Malbec (San Juan)',
 'Chakana 2006 Reserve Malbec (Luján de Cuyo)',
 'Andeluna 2004 Pasionado Red (Uco Valley)',
 'O. Fournier 2005 B Crux Red (Uco Valley)',
 'Ricardo Santos 2006 La Madras Vineyard Malbec (Mendoza)',
 'J. & F. Lurton 2004 Piedra Negra Malbec (Mendoza)',
 'Penedo Borges 2012 Icono Malbec (Luján de Cuyo)',
 'Antigal 2010 One Doña Angeles Vineyard Malbec (Mendoza)',
 'Antigal 2008 One La Dolores Vineyard Malbec (Mendoza)',
 'Bodega Tacuil 2010 33 de Dávalos Malbec (Salta)']

In [ ]:
def recomendar_por_sommelier(sommelier: str, model: LightFM = model_lightfm, features_matrix: csr_matrix = features_matrix, wine_id_to_title: dict = wine_id_to_title, category_mapping: dict = category_mapping, n=10) -> list:
    """
    Recomienda vinos según los mejores puntuados por un sommelier.
    """

    # 1. Obtenemos el ID del sommelier usando el diccionario category_mapping
    sommelier_id = category_mapping["taster_name"][sommelier]

    # 2. Buscamos los vinos puntuados por ese sommelier en features_matrix
    taster_name_index = item_features.columns.get_loc("taster_name")
    wine_ids_sommelier_mask = features_matrix[:, taster_name_index].toarray().flatten() == sommelier_id
    wine_ids_sommelier = item_features.index[wine_ids_sommelier_mask]  # Índices de los vinos puntuados por el sommelier


    # 3. Generamos las predicciones para todos los vinos con ese sommelier
    scores = model.predict(user_ids=0, item_ids=np.arange(features_matrix.shape[0]), item_features=features_matrix)

    # 4. Filtramos por los vinos puntuados por el sommelier
    scores_sommelier = scores[wine_ids_sommelier_mask]

    # 5. Ordenamos las predicciones y seleccionamos los n mejores
    top_items_sommelier = np.argsort(-scores_sommelier)[:n]

    # 6. Obtenemos los nombres de los vinos con su ID
    # Corrección: Obtener los IDs de los vinos recomendados del subconjunto filtrado
    recommendations = [wine_id_to_title[item_features.loc[wine_ids_sommelier[item], 'wine_id']] for item in top_items_sommelier]

    return recommendations


In [ ]:
recomendar_por_sommelier("Kerin O’Keefe")

["Cusumano 2012 Sàgana Tenuta San Giacomo Nero d'Avola (Sicilia)",
 'COS 2013 Frappato (Sicilia)',
 'Feudo Principi di Butera 2012 Symposio Red (Terre Siciliane)',
 'Baglio del Cristo di Campobello 2012 Adènzia Red (Sicilia)',
 'Le Mandolare 2015 Corte Menini  (Soave Classico)',
 'La Mannella 2012  Brunello di Montalcino',
 'Luigi Maffini 2012 Kràtos Fiano (Paestum)',
 'Di Giovanna 2013 Grillo (Terre Siciliane)',
 'Braschi 2013 Il Costone Riserva Sangiovese (Romagna)',
 'Barone di Villagrande 2012 Rosso Villagrande  (Etna)']

In [ ]:
def recomendar_por_variedad(variedad: str, model: LightFM = model_lightfm, features_matrix: csr_matrix = features_matrix, wine_id_to_title: dict = wine_id_to_title, category_mapping: dict = category_mapping, n=10) -> list:
    """
    Recomienda vinos según la variedad de uva.
    """

    # 1. Obtenemos el ID de la variedad usando el diccionario category_mapping
    variedad_id = category_mapping["variety"][variedad]

    # 2. Buscamos los vinos con esa variedad en features_matrix
    variety_index = item_features.columns.get_loc("variety")  # Obtener índice de la columna "variety"
    wine_ids_variedad = features_matrix[:, variety_index].toarray().flatten() == variedad_id  # Buscar ID de la variedad

    # 3. Generamos las predicciones para todos los vinos con esa variedad
    scores = model.predict(user_ids=0, item_ids=np.arange(features_matrix.shape[0]), item_features=features_matrix)

    # 4. Filtramos por los vinos que coincidan con el ID de la variedad
    scores_variedad = scores[wine_ids_variedad]

    # 5. Ordenamos las predicciones y seleccionamos los n mejores
    top_items_variedad = np.argsort(-scores_variedad)[:n]

    # 6. Obtenemos los nombres de los vinos con su ID
    wine_ids_for_variety = item_features.index[wine_ids_variedad]  # obtener los wine_ids que corresponden a la variedad
    recommendations = [wine_id_to_title[item_features.loc[wine_ids_for_variety[item], 'wine_id']] for item in top_items_variedad]

    return recommendations

In [ ]:
recomendar_por_variedad("Pinot Noir")

['Citation 2004 Pinot Noir (Oregon)',
 'Caves Transmontanas 2006 Vértice Pinot Noir (Douro)',
 'Bouchard Père & Fils 2005 Premier Cru  (Pommard)',
 'Henri de Villamont 2005  Grands-Echezeaux',
 'Pali 2006 Fiddlestix Vineyard Pinot Noir (Sta. Rita Hills)',
 'Dunstan 2014 Durell Vineyard Pinot Noir (Sonoma Coast)',
 'August West 2015 Pinot Noir (Santa Lucia Highlands)',
 "Brutocao 2013 Slow Lope'n Vineyard Pinot Noir (Anderson Valley)",
 "River Road 2014 Stephanie's Cuvée Pinot Noir (Green Valley)",
 'Camille Giroud 2008  Côte de Beaune-Villages']